# Warehouse Location

This problem considers the optimal locations to build warehouses meeting customers' demands. Let $N$ be the set of candidate warehouse locations, and $M$ the set of customer locations. Also, we know $d_{nm}$, the cost of delivering product to customer $m$ from warehouse $n$.

The binary variables $y_{n}$ take the value of 1, if warehouse $n$ is selected and 0 otherwise. 
The $x_{nm}$ variables indicate the fraction of the demand for customer $m$ served by warehouse $n$.

Based on the implementation below, could you write down the corresponding algebraic formulation?

In [1]:
from pyomo.environ import *

In [2]:
#Data

N = ['HAR', 'MEM', 'ASX']
M = ['NYC', 'LAX', 'CHI', 'HOU']
d = {('HAR', 'NYC'): 1956, ('HAR', 'LAX'): 1606,\
     ('HAR', 'CHI'): 1410, ('HAR', 'HOU'): 330, \
     ('MEM', 'NYC'): 1096, ('MEM', 'LAX'): 1792,\
     ('MEM', 'CHI'): 531,  ('MEM', 'HOU'): 567, \
     ('ASX', 'NYC'): 485,  ('ASX', 'LAX'): 2322,\
     ('ASX', 'CHI'): 324,  ('ASX', 'HOU'): 1236 }
P = 3

In [3]:
model = ConcreteModel()
model.Locations = N
model.Customers = M

model.x = Var(model.Locations, model.Customers, bounds=(0.0,1.0))
model.y = Var(model.Locations, within = Binary)

In [4]:
model.obj = Objective(expr = sum(d[n,m]*model.x[n,m] for n in model.Locations for m in model.Customers))

In [5]:
model.single_x = ConstraintList()
for m in model.Customers:
    model.single_x.add(sum( model.x[n,m] for n in model.Locations ) == 1.0)
    
model.bound_y = ConstraintList()
for n in model.Locations:
    for m in model.Customers:
        model.bound_y.add(model.x[n,m] <= model.y[n])
        
model.num_facilities = Constraint(expr=sum( model.y[n] for n in model.Locations ) <= P)

In [6]:
solver = SolverFactory('glpk')
solver.solve(model, tee=False)

print()
print('*** Solution *** :')
print()

for i in model.Locations:
    if model.y[i] == 1:
        print("Warehouse at",i, "serving:")
    else:
        continue
    for j in model.Customers:
        if model.x[i,j] != 0:
            print("-->", j, "-- {}%".format(value(model.x[i,j])*100))
    print()
    
print()
print('*** Cost *** :',"$","%.2f" %value(model.obj))
print()

#Another way to display the results
for wl in N:
    if value(model.y[wl]) > 0.5:
        customers = [str(cl) for cl in M if value(model.x[wl, cl]) > 0.5]
        print(str(wl)+" serves customers: "+str(customers))
    else:
        print(str(wl)+": do not build")

print()
print()

print("OPTIMIZATION MODEL DETAILS:")
print()

# This summarizes the information in the Pyomo model, including the constraint and objective expressions.
# It can be a very useful debugging tool when a model is not generating the expected results,
# since it shows the fully expanded version of the model
model.pprint()        


*** Solution *** :

Warehouse at HAR serving:
--> LAX -- 100.0%
--> HOU -- 100.0%

Warehouse at ASX serving:
--> NYC -- 100.0%
--> CHI -- 100.0%


*** Cost *** : $ 2745.00

HAR serves customers: ['LAX', 'HOU']
MEM: do not build
ASX serves customers: ['NYC', 'CHI']


OPTIMIZATION MODEL DETAILS:

6 Set Declarations
    bound_y_index : Dim=0, Dimen=1, Size=12, Domain=None, Ordered=False, Bounds=None
        [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]
    single_x_index : Dim=0, Dimen=1, Size=4, Domain=None, Ordered=False, Bounds=None
        [1, 2, 3, 4]
    x_index : Dim=0, Dimen=2, Size=12, Domain=None, Ordered=False, Bounds=None
        Virtual
    x_index_0 : Dim=0, Dimen=1, Size=3, Domain=None, Ordered=False, Bounds=None
        ['ASX', 'HAR', 'MEM']
    x_index_1 : Dim=0, Dimen=1, Size=4, Domain=None, Ordered=False, Bounds=None
        ['CHI', 'HOU', 'LAX', 'NYC']
    y_index : Dim=0, Dimen=1, Size=3, Domain=None, Ordered=False, Bounds=None
        ['ASX', 'HAR', 'MEM']

2 Var Declarat